In [3]:
!pip install --upgrade mediapipe==0.10.9 opencv-python numpy pygame --quiet

import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)
prev_action = ""

def get_coords(landmarks, idx, shape):
    y = int(landmarks[idx].y * shape[0])
    x = int(landmarks[idx].x * shape[1])
    return x, y

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    action = ""

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        lm = results.pose_landmarks.landmark
        h, w, _ = frame.shape

        # Coordenadas necesarias
        nose_x, nose_y = get_coords(lm, mp_pose.PoseLandmark.NOSE, frame.shape)
        lw_x, lw_y = get_coords(lm, mp_pose.PoseLandmark.LEFT_WRIST, frame.shape)
        rw_x, rw_y = get_coords(lm, mp_pose.PoseLandmark.RIGHT_WRIST, frame.shape)

        lh_x, lh_y = get_coords(lm, mp_pose.PoseLandmark.LEFT_HIP, frame.shape)
        rh_x, rh_y = get_coords(lm, mp_pose.PoseLandmark.RIGHT_HIP, frame.shape)
        lk_x, lk_y = get_coords(lm, mp_pose.PoseLandmark.LEFT_KNEE, frame.shape)
        rk_x, rk_y = get_coords(lm, mp_pose.PoseLandmark.RIGHT_KNEE, frame.shape)

        ls_x, ls_y = get_coords(lm, mp_pose.PoseLandmark.LEFT_SHOULDER, frame.shape)
        rs_x, rs_y = get_coords(lm, mp_pose.PoseLandmark.RIGHT_SHOULDER, frame.shape)

        # Reglas de acción
        if lw_y < nose_y and rw_y < nose_y:
            action = "Brazos arriba"
        elif lh_y > lk_y and rh_y > rk_y:
            action = "Persona sentada"
        elif (
            abs(lw_y - rw_y) < 60 and                           # muñecas a la misma altura
            abs(lw_x - rw_x) < 80 and                           # muñecas muy juntas
            ls_y < lw_y < lh_y and rs_y < rw_y < rh_y          # entre hombros y caderas
        ):
            action = "Brazos cruzados"

        if action and action != prev_action:
            print(action)
            prev_action = action

        if action:
            cv2.putText(frame, action, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow("Reconocimiento de Acciones", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



ERROR: Could not find a version that satisfies the requirement mediapipe==0.10.9 (from versions: 0.10.13, 0.10.14, 0.10.18, 0.10.20, 0.10.21)
ERROR: No matching distribution found for mediapipe==0.10.9


ModuleNotFoundError: No module named 'mediapipe'